In [1]:
import os
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import csv
import json
import sklearn
print(sklearn.__version__)
import pandas as pd
import numpy as np
import sys
from evaluation.metric import computeAccuracy, computeF1Score, computeAUC
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline
from pipelines.ecig import ECigPipeline

0.17


In [2]:
# # test ----------
df_test = pd.read_csv("./data/alcohol_training_instances.csv")
df_test[:3]

,_id,created_at,labels,predict,text,user,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.statuses_count,user.verified
0,556e0ee3d6dfbb462880f0a5,Tue Jun 02 20:16:08 +0000 2015,{'alcohol': 0},0.526050,Impatiently waiting to get our hands on the ne...,{'created_at': 'Thu Jun 12 22:14:05 +0000 2014...,Thu Jun 12 22:14:05 +0000 2014,394,407.0,1997.0,823.0,False
1,556e128ad6dfbb46288111e4,Tue Jun 02 20:31:44 +0000 2015,"{'alcohol': 1, 'first_person': 0, 'alcohol_rel...",0.516649,Beer fans need their @ColumbusBrewing Bodhi. I...,{'created_at': 'Mon Oct 06 21:00:38 +0000 2008...,Mon Oct 06 21:00:38 +0000 2008,806,1006.0,960.0,10442.0,False
2,556e12e6d6dfbb462881153e,Tue Jun 02 20:33:15 +0000 2015,"{'alcohol': 1, 'first_person_level': 0, 'first...",0.523513,Beer Olympics with my #1 fan and babe😍❤️ #Team...,{'created_at': 'Tue Dec 20 02:46:19 +0000 2011...,Tue Dec 20 02:46:19 +0000 2011,11662,563.0,356.0,13940.0,False


In [3]:
# read the data
filePath = "C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol/data/labeled_figure_eight_csv_1000.csv"
df = pd.read_csv(filePath)
df.is_this_posting_relevant_to_ecigarette[:3]

# Deal with ecig_labels.
ecig_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.is_this_posting_relevant_to_ecigarette)):
#     print(df.is_this_posting_relevant_to_ecigarette[i])
    if df.is_this_posting_relevant_to_ecigarette[i] == 'ECIG':
        ecig_labels.append(1)
    elif df.is_this_posting_relevant_to_ecigarette[i] == 'NOT-ECIG':
        ecig_labels.append(0)
    else:
        print('Should not go to here')
# ecig_labels


from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.realdonaldtrump_foxandfriends_smoke_blowing_up_your_orange_ass, ecig_labels, test_size=0.2, random_state=26)
print('length of X_train: ', len(X_train))
print('length of X_test: ', len(X_test))
print('length of y_train: ', len(y_train))
print('length of y_test: ', len(y_test))
# X_train
# y_train


# # test ----------
# df_test = pd.read_csv("./data/alcohol_training_instances.csv")
# df_test
# df_test.labels[1]

# alc_labels = []
# # extracting pre-labeled classification
# for i in range(0,len(df_test.labels)):
#     alc_labels.append(df_test.labels[i][12])
# alc_labels = list(map(int, alc_labels))

# from sklearn.cross_validation import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df_test, alc_labels, test_size=0.1, random_state=26)
# X_train

length of X_train:  801
length of X_test:  201
length of y_train:  801
length of y_test:  201


In [4]:
sum(ecig_labels)

571

In [5]:
# LR
clf_ecig_LR = ECigPipeline().pipeline(LogisticRegression())
clf_ecig_LR.steps

[('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=None, strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

In [6]:
# Fit the data into model to see the performance.

In [7]:
clf_ecig_LR.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
 ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [8]:
# predict in testing set.
y_ecig_LR = clf_ecig_LR.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_ecig_LR, y_test)
f1_score_LR = computeF1Score(y_test, y_ecig_LR)
print('F1 Score: ', f1_score_LR)
auc_score_LR = computeAUC(clf_ecig_LR, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_LR)

length of total comps:  201
62 different item in whole dataset
accuracy: 69.154229%
F1 Score:  0.75
alcohol LR AUC:  0.8499701729966196


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


We can see the results is fine. Now we will use this classifier on our San Francisco data to see how many of them are e-cig related.

In [9]:
type(X_train)

pandas.core.series.Series

In [10]:
SF_filePath = 'C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol/data/juliana_allSF_2018_01.json'
df_SF_01 = pd.read_json(SF_filePath, lines=True, encoding='utf-8') 


In [11]:
df_SF_01[:3]

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id,quoted_status_id_str,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user
0,NaN,None,2018-01-03 03:55:10,"[0, 83]","{'symbols': [], 'urls': [{'expanded_url': 'htt...",NaN,NaN,0,False,low,...,9.482238e+17,9.482238e+17,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",Still totally love my boyfriend even if he can...,2018-01-03 03:55:10.234,False,"{'location': 'Tenderloin, San Francisco', 'def..."
1,NaN,None,2018-01-03 03:17:52,"[17, 47]","{'symbols': [], 'urls': [], 'hashtags': [], 'u...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",@daniellenatoli1 @ me in a pic &amp; then maybe,2018-01-03 03:17:52.590,False,"{'location': 'Brooklyn, New York', 'default_pr..."
2,NaN,None,2018-01-03 03:47:10,"[117, 140]","{'symbols': [], 'urls': [{'expanded_url': 'htt...",NaN,"{'display_text_range': [128, 169], 'entities':...",0,False,low,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/iphone"" r...",@imillhiser @IPM_GS @benwikler @RVAwonk @AdamP...,2018-01-03 03:47:10.777,True,"{'location': 'San Francisco, CA', 'default_pro..."


In [12]:
df_SF_01.text[:3]

0    Still totally love my boyfriend even if he can...
1      @daniellenatoli1 @ me in a pic &amp; then maybe
2    @imillhiser @IPM_GS @benwikler @RVAwonk @AdamP...
Name: text, dtype: object

In [13]:
type(df_SF_01.text)

pandas.core.series.Series

In [14]:
print('length of San Francisco data: ', len(df_SF_01))
y_SF_LR = clf_ecig_LR.predict(df_SF_01.text)
y_SF_LR

length of San Francisco data:  278884


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array([1, 1, 0, ..., 1, 1, 1])

In [15]:
sum(y_SF_LR)

201606

In [16]:
y_SF_LR[:20]

array([1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0])

In [17]:
for text in df_SF_01.text[:20]:
    print(text)
    print('------------')

Still totally love my boyfriend even if he can't ride a bike. We'll take the bus... https://t.co/c1S1OTqPfA
------------
@daniellenatoli1 @ me in a pic &amp; then maybe
------------
@imillhiser @IPM_GS @benwikler @RVAwonk @AdamParkhomenko @SarahLerner @brianbeutler @cristianafarias @jdawsey1… https://t.co/Bf2nsE2POu
------------
@lilnatttt make your mind up, i need you to make your mind up
------------
@NatGeo .@NatGeo's book "Those Inventive Americans" emphasizes how original some human beings are and how their cre… https://t.co/Ojyp84kFGU
------------
te amo te amo te amo https://t.co/anZ0k4S8xy
------------
@her_na_lnr Меня уже ничего не удивляет. Лучше бы легализировали LSD.
------------
Most copped on Restocks last week?
------------
@matthewherper @daviesbj @jgreid @adamfeuerstein @Dr__Guess @YoniFreedhoff And that says a lot
------------
Jeremy Lamb shooting 2/3 from 3pt range. 8 points showing some good spot up shooting.
------------
TEST_LAT/LON: 7f521b31-f8d8-4c2e-85d2-ddb826

In [18]:
clf_ecig_LR.steps

[('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=None, strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

In [19]:
clf_ecig_LR.steps
LR_ecig = clf_ecig_LR.steps[1][1]
# clf_ecig_LR.steps[0][1].tfidf

In [20]:
coefficients = LR_ecig.coef_.tolist()[0]

In [21]:
len(coefficients)

19904

In [22]:
tfidf_ecig = clf_ecig_LR.get_params()['tfidf']
features = tfidf_ecig.get_feature_names()
print('length of features: ', len(features))
type(features)

length of features:  19904


list

In [23]:
featuresAndCoef = {'features':features,'coefficients':coefficients}
df_featuresAndCoef = pd.DataFrame(featuresAndCoef)
df_featuresAndCoef.sort_values(by=['coefficients'], ascending=False)

,features,coefficients
8951,juul,3.204274
17589,vape,2.814391
18016,vaping,1.738162
11014,my,1.016856
11099,my juul,0.839300
12920,pods,0.780632
16015,the juul,0.728589
9073,juul pods,0.634277
10488,me,0.615104
2878,can,0.579114


Even the coefficients seem good, we still got 201606 out of 278884 the classifier said was positive (e-cig relevant), which is not good. How about we include some negative sample?